# Measuring the resolution in an image
Anders Kaestner, Laboratory for Neutron Scattering and Imaging, Paul Scherrer Institut, 2022

<img src='../00_common/figures/by-nc-nd.svg' style='height:30px'/>

## Introduction
The resolution tells the highest spatial frequency you can unambiguously observe in the image. It has an upper limit set by the sampling frequency, but is mostly less. Lower spatial frequencies appear as smooth edges in the image. 

The resolution is a consequence of a series of optical components in your imaging system (penumbra blurring, scintillator, lens, and camera) it could in principle also involve the effect of filtering you applied to the image.

$$h_{system} = h_{penumbra} * h_{scintillator} * h_{lens} * h_{camera} * h_{filters}$$

### What you will learn in this tutorial

### Needed mathematical concepts
- Convolution 
- Fourier transform
- Curve fitting
- Interpolation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import tifffile as tiff
import matplotlib.patches as mpatches
import skimage.filters as flt
import sys
sys.path.append('../00_common/')
import resolutiontools as res

## The principle to measuring the resolution
The resolution can be measured using two main methods
- Using test patterns representing discrete resolutions
- Measuring the width intensity spread from an absorbing edge

<figure>

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,5))
img1 = tiff.imread('../00_common/data/edges/edge20mm_0000.tif')

idx_edge    = 1
idx_pattern = 0
ax[idx_edge].imshow(img1,vmin=300,vmax=30000,cmap='gray');
ax[idx_edge].set_xlabel('x [pixels]')
ax[idx_edge].set_ylabel('y [pixels]');
rect=mpatches.Rectangle((20,200),100,100,fc='r',ec='r',alpha=0.3)
ax[idx_edge].add_patch(rect)

img2 = fits.getdata('data/focus_34mm_60s.fits')
ax[idx_pattern].imshow(img2,vmin=800,vmax=10000,cmap='gray')
ax[idx_pattern].set_title('The PSI test pattern')
bbox_props = dict(boxstyle="round", fc="y", ec="0.5", alpha=1.0)
ax[idx_pattern].text(1150,1100, "1", ha="center", va="center", size=14,
        bbox=bbox_props)
ax[idx_pattern].text(1150,1500, "2", ha="center", va="center", size=14,
        bbox=bbox_props)
ax[idx_pattern].text(1750,950, "2", ha="center", va="center", size=14,
        bbox=bbox_props)
ax[idx_pattern].text(1150,250, "3", ha="center", va="center", size=14,
        bbox=bbox_props)
ax[idx_pattern].set_xlabel('x [pixels]')
ax[idx_pattern].set_ylabel('y [pixels]');

fontsize=14
labelposition= -0.15
# Set label for panel 1 -> (a)
ax[0].text(0.5, labelposition, '(a)', transform=ax[0].transAxes, fontsize=fontsize, ha='center',va='center')  


# Set label for panel 2 -> (b)
ax[1].text(0.5, labelposition, '(b)', transform=ax[1].transAxes, fontsize=fontsize, ha='center',va='center') ; 

<figcaption align = "center"><b>Figure 1</b> - Items to measure the resolution; (a) a printed test pattern and (b) knife edge made of gadolinum (for neutrons)</figcaption>
</figure>

Figure 1 shows two devices that can be used to measure the resolution. Panel (a) shows a segment of a test pattern device with different features to measure the resolution. 
1. Siemens star. The spatial frequeny of the spokes in decrease as function of the radius. 
2. Line patterns. Blocks of parallel line with given spatial frequency.
3. Straight absorbing edges.

The item in panel (b) is an absorbing item with straight edges.

## Measuring the resolution using test patterns

The test pattern method is limited to the spatial frequencies avaliable in the test pattern.

## Measuring the resolution using edge spread
The method of measuring resolution using is based on the extracting the intensity profile across a high contrast sharp edge. The ideal material to produce such an edge in neutron imaging is gadolinium that thanks to its high attenuation coefficient allows to use a thin sheet. The reason for using a thin sheet is that it can lead to errors in the measurement in case the test object is not perfectly perpendicular to the beam. A deviation will add unsharpnes to the measurement due to different material thicknesses as shown in the figure below.

<br/>
<figure>
<img src="figures/materialthickness.svg" style="width:400px"/>
<br/>  
<figcaption align = "center"><b>Figure 2</b> - Edge response with different material thickness may bias the resolution measurements.</figcaption>
</figure>    
<br/>

The test object should also be slightly rotated (3-6$^{\circ}$) in the image plane to provide a slanted edge to allow better edge sampling. The figure below shows the profiles across different positions along the edge. Here, you can see that the shape varies slightly depending on where the profile is extracted.

<figure>

In [ ]:
x,y = np.meshgrid(np.linspace(-10,10,201),np.linspace(-10,10,201))
def sigm(x,s) :
    return 1/(1+np.exp(-x/s))

e = sigm(x+0.0834*y,0.08)
de = np.diff(e)
fig, ax = plt.subplots(1,3,figsize=(15,4))
ax[0].imshow(e,cmap='gray')
ax[0].set_title('Projection of an edge')
ax[1].imshow(de)
ax[1].set_title('Derivative of the edge')
begin=80
end=120
alpha=1
a = 33
b = 100
c = 185

ax[1].hlines([a,b,c],xmin=begin,xmax=end,color='red')
bbox_props = dict(boxstyle="round", fc="y", ec="0.5", alpha=1.0)
ax[1].text(end+20,a, "A", ha="center", va="center", size=14,
        bbox=bbox_props)
ax[1].text(end+20,b, "B", ha="center", va="center", size=14,
        bbox=bbox_props)
ax[1].text(end+20,c, "C", ha="center", va="center", size=14,
        bbox=bbox_props)

pA = de[a, begin:end]
pB = de[b,begin:end]
pC = de[c,begin:end]


ax[2].stairs(pA,label='A',alpha=alpha)
ax[2].stairs(pB,label='B',alpha=alpha)
ax[2].stairs(pC,label='C',alpha=alpha)
ax[2].legend()
ax[2].set_title('Profiles at different edge positions');

<figcaption align = "center"><b>Figure 3</b> - An edge and its profiles. The profile shape varies with the position.</figcaption>
</figure>

### Extracting the edge profile
We need to sample several positions along the edge to take advantage of the slanted edge and even achive subpixel accuracy. This must be done carefully using the distance to the edge. There are different methods to do the edge extraction. Among others thresholding and using the distance transform. Here, we will use the method described in the pixel size tutorial to fit a straight line to the edge and compute the average intensity at each distance from the edge.

In [ ]:
res.

## Summary